# Module 5 - Modern Portfolio Theory

In this module, We’ll be looking at investment portfolio optimization with python, the
fundamental concept of diversification and the creation of an efficient frontier that can be used by investors to choose
specific mixes of assets based on investment goals; that is, the trade off between their desired level of portfolio
return vs their desired level of portfolio risk.

[Modern Portfolio Theory](https://www.investopedia.com/terms/m/modernportfoliotheory.asp) suggests that it is possible to 
construct an "efficient frontier" of optimal portfolios,
offering the maximum possible expected return for a given level of risk. It suggests that it is not enough to look at
the expected risk and return of one particular stock. By investing in more than one stock, an investor can reap the
benefits of diversification, particularly a reduction in the riskiness of the portfolio. MPT quantifies the benefits of
diversification, also known as not putting all of your eggs in one basket.

In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
import glob
import random

## Problem Statements

## 5.1 
For your chosen stock, calculate the mean daily return and daily standard deviation of returns, and then just annualise them to get mean expected annual return and volatility of that single stock. ( annual mean = daily mean * 252 , annual stdev = daily stdev * sqrt(252) )

In [2]:
def read_csv( filename ):
    if isinstance(filename, pd.DataFrame): return filename  # OPTIMIZATION: allow passthrough of existing dataframe
    
    dataframe = pd.read_csv( filename, parse_dates=['Date'] )
    dataframe.set_index( dataframe.Date, inplace=True )
    return dataframe

def meanDailyReturn( filename ):
    return read_csv( filename ).Close_Price.pct_change().dropna().mean()

def meanDailySTD( filename ):
    return read_csv( filename ).Close_Price.pct_change().dropna().std()

def meanAnnualReturn( filename ):
    return meanDailyReturn(filename) * 252

def meanAnnualSTD( filename ):
    return meanDailySTD(filename) * np.sqrt(252)

def getName( filename ):
    return re.sub(r'^.+/|\.[^.]+$',     '',    filename)

def getCap( filename ):
    return re.sub(r'^.*/(\w+_Cap)/.*$', '\\1', filename)

def calcReturnVolatility( filename ):
    input  = read_csv( filename )
    output = DataFrame([{
        "Name":             getName( filename ),
        "Cap":              getCap(  filename ),
        "meanDailyReturn":  meanDailyReturn( input ),
        "meanDailySTD":     meanDailySTD( input ),
        "meanAnnualReturn": meanAnnualReturn( input ),
        "meanAnnualSTD":    meanAnnualSTD( input )
    }])
    output.set_index( output.Name, inplace=True, drop=False )
    return output

stock = '../../data_output/module_1/python3/stocks/Large_Cap/ADANIPORTS.csv'
calcReturnVolatility(stock)

,Cap,Name,meanAnnualReturn,meanAnnualSTD,meanDailyReturn,meanDailySTD
Name,,,,,,
ADANIPORTS,Large_Cap,ADANIPORTS,0.060499,0.309751,0.00024,0.019512


## 5.2 
Now, we need to diversify our portfolio. Build your own portfolio by choosing any 5 stocks, preferably of different sectors and different caps. Assume that all 5 have the same weightage, i.e. 20% . Now calculate the annual returns and volatility of the entire portfolio ( Hint : Don't forget to use the covariance )

In [3]:
filenames = glob.glob('../../data_output/module_1/python3/stocks/**/*.csv')
filenames = random.sample( filenames, 5 )
summary   = pd.concat([ calcReturnVolatility(stock) for stock in filenames ])
summary

,Cap,Name,meanAnnualReturn,meanAnnualSTD,meanDailyReturn,meanDailySTD
Name,,,,,,
NBCC,Mid_Cap,NBCC,-0.515176,0.508672,-0.002044,0.032043
IOC,Large_Cap,IOC,-0.413701,0.500046,-0.001642,0.031500
TCS,Large_Cap,TCS,0.075027,0.431617,0.000298,0.027189
ADANIPORTS,Large_Cap,ADANIPORTS,0.060499,0.309751,0.000240,0.019512
AJANTPHARM,Mid_Cap,AJANTPHARM,-0.173891,0.332171,-0.000690,0.020925


In [4]:
portfolio = DataFrame()
for filename in filenames:
    portfolio[ getName(filename) ] = read_csv(filename).Close_Price
portfolio.head()

,NBCC,IOC,TCS,ADANIPORTS,AJANTPHARM
Date,,,,,
2017-05-15,197.60,442.10,2365.10,356.20,1633.50
2017-05-16,198.25,446.60,2429.15,354.50,1634.25
2017-05-17,200.35,444.25,2455.35,348.55,1654.35
2017-05-18,194.20,439.90,2536.20,347.90,1633.40
2017-05-19,193.55,435.40,2507.15,349.45,1670.25


In [5]:
# Portfolio Mean Average Return can be calculated either from the summary data or the portfolio table
portfolio_annual_returns = summary.meanAnnualReturn.mean()
portfolio_annual_returns

-0.19344865253567306

In [6]:
# NOTE: mean().mean() works because weights between each of the stocks is equal
portfolio_annual_returns = portfolio.pct_change().mean().mean() * 252  
portfolio_annual_returns

-0.19344865253567306

In [7]:
# Portfolio Covarence matrix
portfolio.pct_change().cov()

,NBCC,IOC,TCS,ADANIPORTS,AJANTPHARM
NBCC,1.026776e-03,0.000159,-2.158218e-07,0.000112,0.000153
IOC,1.587470e-04,0.000992,-1.622964e-05,0.000077,0.000042
TCS,-2.158218e-07,-0.000016,7.392593e-04,-0.000038,0.000033
ADANIPORTS,1.122036e-04,0.000077,-3.775636e-05,0.000381,0.000100
AJANTPHARM,1.534440e-04,0.000042,3.348475e-05,0.000100,0.000438


In [8]:
# Now calculate the annual volatility
weights = np.full( portfolio.shape[1], 1/portfolio.shape[1] )
weights

array([0.2, 0.2, 0.2, 0.2, 0.2])

In [11]:
portfolio_annual_volatility = np.sqrt(
    np.dot( 
        weights.T, 
        np.dot( portfolio.pct_change().cov(), weights )
    ) 
    * np.sqrt(252)
)
round( portfolio_annual_volatility, 2 )

0.06

In [12]:
print("Portfolio Annualized Mean Return: ", round(portfolio_annual_returns,2)    )
print("Portfolio Annualized Volatility:  ", round(portfolio_annual_volatility,2) )

Portfolio Annualized Mean Return:  -0.19
Portfolio Annualized Volatility:   0.06


## 5.3 
Prepare a scatter plot for differing weights of the individual stocks in the portfolio , the axes being the returns and volatility. Colour the data points based on the Sharpe Ratio ( Returns/Volatility) of that particular portfolio.

# 5.4 Mark the 2 portfolios where:
- Portfolio 1 - The Sharpe ratio is the highest 
- Portfolio 2 - The volatility is the lowest.